## asdasd

In [1]:
import torch

/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ordinaloss_distributed.utils import data_utils

/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded ordinaloss_distributed.utils.data_utils


RuntimeError: Default process group has not been initialized, please make sure to call init_process_group.

In [11]:
import torch
from ordinaloss_distributed.engine.model_engine import OrdinalEngine, ddp_setup
from ordinaloss.utils.pretrained_models import classification_model_vgg, DummyModel
from ordinaloss_distributed.utils.data_utils import data_load
from torch.optim import SGD
from torch.distributed import destroy_process_group

In [2]:
from ordinaloss.utils.loss_utils import CSCELoss
from ordinaloss.utils.loss_utils import create_ordinal_cost_matrix

loaded ordinaloss.utils.loss_utils


In [16]:
def main(device:int, world_size:int, total_epochs):
    ddp_setup(device, world_size=world_size)
    print("buga")

    META_LEARNING_RATE = 10
    LEARNING_RATE = 1.0e-3
    WEIGHT_DECAY = 5.0e-2
    COST_DISTANCE = 3
    DIAGONAL_VALUE = 20
    PATIENCE = 3
    MIN_DELTA = 0.99
    model = DummyModel()
    loaders, _, _ = data_load("../datasets/kneeKL224/", 16)
    engine = Trainer(
        model=model, loss_fn=None, #The loss function would be modified soon.
        device=device, loaders=loaders, n_classes=5,
        use_lr_scheduler=False, callbacks=[],
        optimizer_fn=SGD, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
        )
    cost_matrix = create_ordinal_cost_matrix(5, cost_distance=COST_DISTANCE, diagonal_value=DIAGONAL_VALUE)
    csce_loss = CSCELoss(cost_matrix)
    engine.set_loss_fn(csce_loss) #The loss is assigned again
    engine.train_until_converge() 
    destroy_process_group()

In [17]:
import torch.multiprocessing as mp

world_size = torch.cuda.device_count()
mp.spawn(main2, args=(world_size,20), nprocs=world_size)


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'main2' on <module '__main__' (built-in)>


ProcessExitedException: process 3 terminated with exit code 1